# Settings

In [1]:
import pandas as pd
import numpy as np
import codecs
import pickle
import random
import math
import re
import ast
import sys

np.set_printoptions(threshold=sys.maxsize)

In [2]:
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#real dataset
datapath = './dataset/dataset-quran.xlsx'

In [4]:
#concept path buat di drive
conceptpath = './assets/conceptInDrive.xlsx' 

In [5]:
### for real data ###
TfResultPath = "./assets/TfResultPath.bin.txt"
DfResultPath = "./assets/DfResultPath.bin.txt"
TfIdfResultPath = "./assets/TfIdfResultPath.bin.txt"

In [6]:
# leskOutputPath = "./assets/realTrainLeskModel_1_100.bin.txt"

# leskOutputPath = "./assets/realTrainLeskModel_100_200.bin.txt"

# leskOutputPath = "./assets/realTrainLeskModel_200_300.bin.txt"

In [7]:
# leskOutputPath = "./assets/realTrainLeskModel_124.bin.txt"


In [8]:
wordembedLeskPath = "./assets/realTrainWordembedModel.bin.txt"

In [9]:
# Running Ke-1
startAyat = 0
endAyat = 100

# Running Ke-2
# startAyat = 100
# endAyat = 200

# Running Ke-3

# Running Ke-4

# Common Function

In [10]:
def readFileBin(path = ''):
    file = open(path, 'rb')
    object_file = pickle.load(file)
    return object_file

In [11]:
'''
Input   : -
Output  : list of terms
Problem : ambil terms wiki di  google sheet
'''
def getListTermsFromSheet(sheetname, datapath = datapath):
    dfUWords = pd.read_excel(datapath, sheet_name=sheetname)
    # Convert word false and true as str
    booldict = {True: 'true', False: 'false'}
    dfUWords = dfUWords.replace(booldict)
    return dfUWords.values.ravel()

def getTerms():
    # ambil data terms dari drive
    listTerms= getListTermsFromSheet('wiki-unique-words')
    return listTerms

In [12]:
def getConcepts():
    dfConcept = pd.read_excel(conceptpath)
    return dfConcept

In [13]:
# definisikan concepts
dfConcept = getConcepts()
# definisikan terms dengan --- function getTerms() ---
listTerms = getTerms().tolist()

# TF dan IDF 

```Dilakukan di spyder, karena memory ga kuat kalo pake data concepts yang pake excell```

**INI UNTUK TF-IDF**

In [14]:
def get_tf(path):
    tf = readFileBin(path)
    return tf

def get_df(path):
    df = readFileBin(path)
    return df

In [15]:
#*** ubah path ini kalo dipake buat data real
tf = get_tf(TfResultPath)
tf = tf.astype('uint8')
#*** ubah path ini kalo dipake buat data real
dfDocFreq = get_df(DfResultPath)
dfDocFreq = dfDocFreq.astype('float16')
# docs = tf.columns

In [16]:
'''
Desc    : IDF
Input   : -
Output  : matrix idf. Fungsi ini buat tf-idfnya lesk algorithm yag udah ditambah overlaps
DONE Date: 21/10/2020
'''
# # INI YANG BARU DIUBAAAAAHAHHHHH
# def tf_idf(dfTfWithOverlaps, dfDocFreq):
#     # get terms
#     terms = dfTfWithOverlaps.index
#     # get docs
#     docs  = dfTfWithOverlaps.columns
#     # create dataframe for tfidf
#     dfTfIdf = dfTfWithOverlaps.copy()
    
#     for i, doc in enumerate(docs):
#         for j, term in enumerate(terms):
#             # overwrite tfidf dataframe dengan hasil perhitungan tfidf
#             dfTfIdf.loc[term, doc] = dfTfWithOverlaps.loc[term, doc]*dfDocFreq.loc[term, 'docfreq']
   
#     return dfTfIdf

# INI YANG BARU DIUBAAAAAHAHHHHH

# dictTermsWindowingConcepts = {
#     "name": [1,2,3],
#     "allah": [2, 4, 5],
#     "beneficent": [1, 2, 3]
# }

def tf_idf(dictTermsWindowingConcepts):
    listOfTerms = dfDocFreq.index
    # Iterating over keys 
    for state in dictTermsWindowingConcepts: 
        if state in listOfTerms:
            dictTermsWindowingConcepts[state] = np.array(dictTermsWindowingConcepts[state])*dfDocFreq.loc[state, 'docfreq']
        else:
            # kalo wordnya ga ada, defaultnya 1
            dictTermsWindowingConcepts[state] = np.array(dictTermsWindowingConcepts[state])* math.log10(40147/1)
    return dictTermsWindowingConcepts

# Lesk Algorithm


**Struktur Function di Lesk Algorithm**
```
- func-leskAlgorithm
    |
    - func-weighting
        |
        - (inisiasi dataframe term-by-concept)
        - func-windowing
        - func-termByConceptMatrixWeighted
            |
            - func-tokenizeWordnetDefinition
            - func-countOverlapDefinition
            - func-countOverlapsConcept
            - func-calculateTargetTermConceptVector
            - func-insertTargetWordVectorToDf
```



### Weighting terms by concepts

```
# Ada line yang harus dihapus untuk data yang real
```

In [17]:
'''============ 1. ============'''
'''
Input   : insert vector target word ke matriks dataframe ayat
Output  : matriks dataframe ayat
Problem : -
DONE Date: 21/10/2020
''' 
def insertTargetWordVectorToDf(dfTermsConcepts, dictTargetWordVector):
    # ambil target word sembari hapus key-nya
    targetWord = dictTargetWordVector.pop('target_kata', None)
    # dict dijadikan array - mempermudah looping
    dictItems = dictTargetWordVector.items()
    
    # looping tiap items dict target word
    for i, item in enumerate(dictItems):
        # ambil pageName dari tuple item
        pageName = item[0]
        # if pageName in dfTermsConcepts.columns:
        dfTermsConcepts.loc[targetWord, pageName] = dictTargetWordVector[pageName]
        # else:
            # dfTermsConcepts.loc[targetWord, pageName] = 0.0
        # print('INSERTED : ', i, ' / ', dfTermsConcepts.loc[targetWord, pageName])
    
    return dfTermsConcepts




'''============ 2. ============'''
'''
Input   : hitung concept vector untuk target word
Output  : Vector of target terms by concepts
Problem : -
DONE Date: 21/10/2020
''' 
# INI YANG BARU DIUBAAAAAHAHHHHH
def calculateTargetTermConceptVector(dictNormalizedWindowingConcepts, targetWord):
    # milai concept windows
    cw = len(dictNormalizedWindowingConcepts)
    # inisiasi untuk target vector
    vectorTargetWord = []

    for i, state in enumerate(dictNormalizedWindowingConcepts):
        if i == 0:
            vectorTargetWord = dictNormalizedWindowingConcepts[state]
        else:
            vectorTargetWord = np.add(vectorTargetWord, dictNormalizedWindowingConcepts[state])
    
#     print(targetWord, "\n")
#     display(vectorTargetWord)
    
    return vectorTargetWord



'''============ 3. ============ 
@PERBAIKI, yang ditambah cuma target wordnya aja
@DONE
'''
'''
Input   : - windowingWords
Output  : Vector of terms by concepts
Problem : - ngitungnya
DONE Date: 21/10/2020
''' 
# INI YANG BARU DIUBAAAAAHAHHHHH
def countOverlapsConcept(windowingWords, countOverlapsContext):
    listOfPages = tf.columns
    
    listOfTerms = tf.index
    
    dictTermsWindowingConcepts = {}
    
#     dfTermsWindowingConcepts = pd.DataFrame(0, index=windowingWords, columns=listOfPages)

#     print("countOverlapsContext : ", countOverlapsContext)
    for word in windowingWords:        
        windowWordVector = []
        for page in listOfPages:
            # Cell diisi jumlah term frequency + jumlah overlaps
            if word in listOfTerms:
                windowWordVector.append(tf.loc[word, page] + next(item for item in countOverlapsContext if item["word"] == word)['overlap'])
            else:
                windowWordVector.append(0 + next(item for item in countOverlapsContext if item["word"] == word)['overlap'])
                #             dfTermsWindowingConcepts.loc[word, page] = tf.loc[word, page] + next(item for item in countOverlapsContext if item["word"] == word)['overlap']
        dictTermsWindowingConcepts[word] = windowWordVector
    
# #     data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}
#     dfTermsWindowingConcepts = pd.DataFrame.from_dict(dictTermsWindowingConcepts, orient='index')
    # Normalize Matriks Term Frequency using IDF
#     dfNormalizedWindowingConcepts = tf_idf(dfTermsWindowingConcepts, dfDocFreq)
    dictNormalizedWindowingConcepts = tf_idf(dictTermsWindowingConcepts)
    
#     #******* PRINT
#     display(dfNormalizedWindowingConcepts)

    return dictNormalizedWindowingConcepts




'''============ 4. ============ 
@Perbaiki Perhitungan Overlaps Pada WordNet-nya (liat paper rujukan)
@DONE'''
'''
Input   : - hasil ravel of list of tokenization definition from wordnet
Output  : Count of overlap
Problem : - cara ngitungnya gimana
''' 
def countOverlapDefinition(windowingWords, windowingWordsDef):
    windowOverlaps = []
    for word in windowingWords:
        # nntinya buat dihapus bergantian, sesuai word saat ini
        listWindowingWords = windowingWords.tolist().copy()
        # ambil element windowingDef berdasarkan kata saat ini
        wordDef = next(item for item in windowingWordsDef if item["word"] == word)
        # hitung overlaps concept window dengan definisi kata-nya
        countOverlap = 0
        listWindowingWords.remove(word)
        for el in listWindowingWords:
            # cari string word di definition, ambil panjang arraynya
            countSearchRegex = len(re.findall("{}".format(el), wordDef['def']))
            countOverlap += countSearchRegex
        windowOverlaps.append({
            "word": word,
            "overlap": countOverlap
        })
    return windowOverlaps




'''============ 5. ============ 
@PERBAIKI, Kita Harus tau mana definisi untuk tiap kata di windowingnya - simpen di dict
@DONE
'''
'''
Input   : - word
Output  : hasil ravel of list of tokenization definition from wordnet
Problem : - remove punct
          - return ravel dari list of definition of word in tokenization
'''   
def tokenizeWordnetDefinition(word):
    listOfDefinition = []
    # ngambil list definition dari suatu word
    definitions = wn.synsets(word)
    for d in definitions:
        # clean def dari non-alphabet
        strDef = re.sub('[^A-Za-z]+',' ', d.definition())
        listOfDefinition.append(strDef)
    # join semua definisi ke satu str
    ravelListOfDefinition = " ".join(listOfDefinition)
    return ravelListOfDefinition

### Windowing and Main Weighting

In [18]:
'''============ 6. ============'''
'''
Input   : - matrix zero dengan index terms dan kolom concepts
          - windowing words
          - target word
Output  : matrix terms-by-concepts yang sudah ada weight-nya dari target word pada suatu ayat
Problem : - weighting dengan wordnet
          - hitung overlapping dengan lesk algorithm
          - normalize dengan tf-idf
'''     
def termByConceptMatrixWeighted(windowingWords, targetWord):
    # Count Overlaps Context
    windowingWordsDef = []
    for w in windowingWords:
        # tokenizedDef harusnya bentuknya dict, nyimpe wordnya apa, dan token definisinya
        tokenizedDef = tokenizeWordnetDefinition(w)
        wordDef = {
            "word": w,
            "def": tokenizedDef
        }
        windowingWordsDef.append(wordDef)

    # print("## WORD DEF ", windowingWordsDef)
        
    # Count Overlaps WordNet
    # windowingWordsDef = [{'word': 'name', 'def': 'merciful beneficent beneficent a language unit by which a person or thing is known a person s reputation family based on male descent a well known or notable person by the sanction or authority of a defamatory or abusive word or phrase assign a specified usually proper proper name to give the name or identifying characteristics of refer to by name or some other identifying characteristic property charge with a function charge to be create and charge with a task or function mention and identify by name make reference to identify as in botany or biology for example give or make a list of name individually give the names of determine or distinguish the nature of a problem or an illness through a diagnostic analysis'}, {'word': 'allah', 'def': 'Muslim name for the one and only God'}, {'word': 'beneficent', 'def': 'doing or producing good generous in assistance to the poor'}, {'word': 'merciful', 'def': 'showing or giving mercy  used conventionally of royalty and high nobility gracious allah'}]
    # windowingWords = ['name', 'allah', 'beneficent', 'merciful']
    print('       - Count Overlaps...')
    countOverlapsContext = countOverlapDefinition(windowingWords, windowingWordsDef)
    # print(countOverlapsContext)
        
    print('       - Normalized Concept...')
    # Count Overlaps Concept
    dictNormalizedWindowingConcepts = countOverlapsConcept(windowingWords, countOverlapsContext)
    
#     print(dictNormalizedWindowingConcepts)
       
    print('       - Target Word Vector...')
    # hitung vector term concept untuk target word
    # return isi vector buat yang target word aja dalam bentuk dictionary
    targetWordVector = calculateTargetTermConceptVector(dictNormalizedWindowingConcepts, targetWord)
    
    #******* PRINT
#     print("TARGET WORD VECTOR : ", targetWordVector)

        
#     print('       - Insert Target Word Vector...')
#     # function buat masukin vector target word ke dataframe terms concept ayat
#     dfTermsConcepts = insertTargetWordVectorToDf(dfTermsConcepts, dictTargetWordVector)
#     dfTermsConcepts
    
    return targetWordVector


In [19]:
'''============ 7. ============'''
'''
Input   : jumlah windowing, ayat dan target word (indexnya)
Output  : return words berdasarkan windowsnya
Problem : - untuk data yang kurang dari jumlah window yang saat itu digunakan ? return semuanya
          - cek dengan print dokumen dummy
          - gimana kalo posisi target ada di (awal, tengah, ujung)
'''
def windowing(idxTarget, tokenAyat, n_window):
    # 0 kalau ada minus
    left = 0 if((idxTarget - n_window) < 0) else (idxTarget - n_window)
    # len target kalau lebih dari length nya
    right = (len(tokenAyat) - 1) if((idxTarget + n_window) > (len(tokenAyat) - 1)) else (idxTarget + n_window)
    
    # ambil kata-kata di kiri
    leftWords = tokenAyat[left : idxTarget]
    # ambil kata-kata dikanan
    rightWords = tokenAyat[idxTarget+1 : right + 1]

    # gabungkan jadi satu array
    windowingWords = [tokenAyat[idxTarget]] + leftWords + rightWords
    
    # ========= PERUBAHAN PENTING NIH ===========
    # delete word yang duplikat ketika berada dalam satu window
    # kasusnya ayat ke-5 word "thee" 
    windowingWordsClean = np.unique(np.array(windowingWords))
    
    #print('{} - {} - target : {} '.format(leftWords, rightWords, tokenAyat[idxTarget]))
    return windowingWordsClean

### Child of Lesk Algorithm



```
# Ada line yang harus dihapus untuk data yang real
```



In [20]:
'''============ 8. ============'''
'''
Input   : token ayat dan jumlah windowing
Output  : matrix terms-by-concepts dari suatu ayat
Problem : - jumlah overlaps menggunakan wordNet 
          - jumlah kata di page, sesuai dengan jumlah konsepnya
          - jumlah setiap term muncul di beberapa dokumen wikipedia (DF)
'''
# INI BARU DIUBAAAAAAAAAAAAAHHHH
def weighting(tokenAyat, n_window):    
    '''Bikin list page yang ada di dummy, delete line ini kalo buat real'''
#     pageDummyList = []
#     for pageName in dfConcept.name:
#     if re.split("\_", pageName)[0] in listTerms:
#         pageDummyList.append(pageName)

    print('- Creating DataFrame Terms Concept...')
    # # inisialisasi matrix kosongnya dulu untuk matriks terms-by-concept, untuk yang real
    dfTermsConcepts = pd.DataFrame(0, index=list(range(len(dfConcept.name))) , columns=listTerms)
#     dfTermsConcepts = pd.DataFrame(0, index=listTerms, columns=dfConcept.name)
    dictTermsConcepts = {}

    '''pake column yang dummy, delete line ini kalo buat real'''
#     dfTermsConcepts = pd.DataFrame(0, index=listTerms, columns=pageDummyList)


    print('- Windowing & Weighting...')
    # looping for every word as target words
    for idxTarget, targetWord in enumerate(tokenAyat):
        print('  # Terget Word {}/{}'.format(idxTarget, len(tokenAyat)-1))
        print('    Target Word : {}'.format(targetWord))
        # --- function windowing() ---
        print('    - Windowing...')
        windowingWords = windowing(idxTarget, tokenAyat, n_window)
        print('      Windowing Words : {}'.format(windowingWords))
        # --- function termByConceptMatrixWeighted() ---
        print('    - Weighting...')
        # looping recursive buat ngisi nilai si dfTermsConcepts-nya
        targetWordVector = termByConceptMatrixWeighted(windowingWords, targetWord)

        dictTermsConcepts[targetWord] = targetWordVector
        # #*** Nyoba 1 target word
        #return dfTermsConcepts

#     print("dictTermsConcepts : ", dictTermsConcepts)
    
    # ===== CATATAAAAAN ====
    # replace column di dictTermsConcepts yang ada di token ayat
    for term in dictTermsConcepts:
        # dfTermsConcepts.iloc[listTerms.index(term)] = dictTermsConcepts[term]
        dfTermsConcepts[term] = dictTermsConcepts[term]
    
    # Rubah dtypes biar ga terlalu gede sizenya
#     dfTermsConcepts = dfTermsConcepts.astype('uint8')
    dfTermsConcepts = dfTermsConcepts.astype('float16')
    
#     display(dfTermsConcepts)
      
    return dfTermsConcepts 

    # Lakukan proses weighting disetiap pages untuk setiap termsnya
    # weight untuk target word (rumusnya ada di dokumen / catetan)
    # setiap term akan memiliki weight dalam vector (concept vector) dengan panjang |wiki pages|

### Save Lesk to .bin

In [23]:
'''============ 10. ============'''
'''
Input   : term by concept matrices
Output  : saved model in .bin.txt
Problem : -
DONE Date: 22/10/2020
''' 
def saveTermsByConceptMatrices(idx, listWeightedAyat, leskOutputPath = './assets/leskOutput/ayat'):
    with codecs.open("{}_{}".format(leskOutputPath, idx), 'wb') as outfile:
        pickle.dump(listWeightedAyat, outfile) 


### Main Lesk Algorithm

```
# Ada line yang harus dihapus untuk data yang real
```

In [24]:
'''============ 9. ============'''
'''
Input   : file dataset
Output  : matrices of terms-by-concept sebanyak documents
Problem : - looping tiap ayat (doc)
          - untuk setiap ayatnya, lakukan weighting oleh def yg lain
'''
def leskAlgorithm(n_window = 2):
    # declare list buat nampung ayat yang udah di weighted
    listWeightedAyat = []
    
    # AYAT YANG INGIN DIGUNAKAN
    # ambil data dari drive
    ayatInString = pd.read_excel(datapath, sheet_name='proceed-data').Terjemahan.values[startAyat : endAyat]
    labelsAllAyat = pd.read_excel(datapath, sheet_name='proceed-data').iloc[:,4:20].values[startAyat : endAyat]
    
    
    # Looping datanya
    for idx, ayat in enumerate(ayatInString):
        # create dictionary for every ayat
        dictPerAyat = dict()

        # insert label ayat to dict
        dictPerAyat["labelsPerAyat"] = np.uint8(labelsAllAyat[idx]) # labelsAllAyat[idx]

        print('==== AYAT - {}/{} ===='.format(idx, len(ayatInString)-1))
        # ubah string ke bentuk array
        tokenAyat = ast.literal_eval(ayat)

        # --- weighting pake function weighting() ---
        print('# Lesk Algorithm...')
        weightedAyat = weighting(tokenAyat, n_window)

        # insert weighted ayat ke dictionary per ayat
        # dibalik column dan indexnya
        dictPerAyat["weightedAyat"] = weightedAyat

        # append weighted ayat ke list
        # listWeightedAyat.append(dictPerAyat)
        
        
        saveTermsByConceptMatrices(idx, dictPerAyat, leskOutputPath = './assets/leskOutput/ayat')
        

        #*** Nyoba 1 Ayat
#         print('- Weighted Ayat with Lesk...')
#         with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#             display(weightedAyat)
#         return weightedAyat
  
    return 0 # listWeightedAyat



#     idx = 24

#     ayat = ayatInString[idx]
#     # create dictionary for every ayat
#     dictPerAyat = dict()

#     # insert label ayat to dict
#     dictPerAyat["labelsPerAyat"] = labelsAllAyat[idx]
    
#     print(np.uint8(labelsAllAyat[idx]))
#     print(labelsAllAyat[idx])

#     print('==== AYAT - {}/{} ===='.format(idx, len(ayatInString)-1))
#     # ubah string ke bentuk array
#     tokenAyat = ast.literal_eval(ayat)

#     # --- weighting pake function weighting() ---
#     print('# Lesk Algorithm...')
#     weightedAyat = weighting(tokenAyat, n_window)

#     # insert weighted ayat ke dictionary per ayat
#     # dibalik column dan indexnya
#     dictPerAyat["weightedAyat"] = weightedAyat

#     # append weighted ayat ke list
#     listWeightedAyat.append(dictPerAyat)
  
#     print(listWeightedAyat)
#     listWeightedAyat

### Running Lesk 

In [25]:
n_window = 2
leskAlgorithm(n_window)
# listWeightedAyat = leskAlgorithm(n_window)
# saveTermsByConceptMatrices(listWeightedAyat) ### ubah path yang lesk outputnya
# listWeightedAyat

==== AYAT - 0/99 ====
# Lesk Algorithm...
- Creating DataFrame Terms Concept...
- Windowing & Weighting...
  # Terget Word 0/3
    Target Word : name
    - Windowing...
      Windowing Words : ['allah' 'beneficent' 'name']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 1/3
    Target Word : allah
    - Windowing...
      Windowing Words : ['allah' 'beneficent' 'merciful' 'name']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 2/3
    Target Word : beneficent
    - Windowing...
      Windowing Words : ['allah' 'beneficent' 'merciful' 'name']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 3/3
    Target Word : merciful
    - Windowing...
      Windowing Words : ['allah' 'beneficent' 'merciful']
    - Weighting...
       - Count Overlaps...
       - Normalized Co

       - Target Word Vector...
  # Terget Word 10/12
    Target Word : brought
    - Windowing...
      Windowing Words : ['astray' 'brought' 'go' 'thy' 'wrath']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 11/12
    Target Word : go
    - Windowing...
      Windowing Words : ['astray' 'brought' 'go' 'wrath']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 12/12
    Target Word : astray
    - Windowing...
      Windowing Words : ['astray' 'brought' 'go']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
==== AYAT - 7/99 ====
# Lesk Algorithm...
- Creating DataFrame Terms Concept...
- Windowing & Weighting...
  # Terget Word 0/2
    Target Word : alif
    - Windowing...
      Windowing Words : ['alif' 'lam' 'mim']
    - Weighting...
       - Count Overlaps...
       - Normalize

       - Target Word Vector...
  # Terget Word 6/6
    Target Word : believe
    - Windowing...
      Windowing Words : ['believe' 'warn']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
==== AYAT - 13/99 ====
# Lesk Algorithm...
- Creating DataFrame Terms Concept...
- Windowing & Weighting...
  # Terget Word 0/10
    Target Word : allah
    - Windowing...
      Windowing Words : ['allah' 'seal' 'set']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 1/10
    Target Word : set
    - Windowing...
      Windowing Words : ['allah' 'seal' 'set' 'upon']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 2/10
    Target Word : seal
    - Windowing...
      Windowing Words : ['allah' 'heart' 'seal' 'set' 'upon']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...


       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 1/6
    Target Word : make
    - Windowing...
      Windowing Words : ['land' 'make' 'mischief' 'said']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 2/6
    Target Word : mischief
    - Windowing...
      Windowing Words : ['land' 'make' 'mischief' 'said' 'say']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 3/6
    Target Word : land
    - Windowing...
      Windowing Words : ['land' 'make' 'mischief' 'peace' 'say']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 4/6
    Target Word : say
    - Windowing...
      Windowing Words : ['land' 'maker' 'mischief' 'peace' 'say']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
      

       - Target Word Vector...
  # Terget Word 3/9
    Target Word : back
    - Windowing...
      Windowing Words : ['back' 'leaf' 'mockery' 'pay' 'shall']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 4/9
    Target Word : mockery
    - Windowing...
      Windowing Words : ['alone' 'back' 'leaf' 'mockery' 'pay']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 5/9
    Target Word : leaf
    - Windowing...
      Windowing Words : ['alone' 'back' 'inordinacy' 'leaf' 'mockery']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 6/9
    Target Word : alone
    - Windowing...
      Windowing Words : ['alone' 'blindly' 'inordinacy' 'leaf' 'mockery']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget W

...

       - Target Word Vector...
  # Terget Word 7/18
    Target Word : wish
    - Windowing...
      Windowing Words : ['enter' 'food' 'gate' 'wherever' 'wish']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 8/18
    Target Word : enter
    - Windowing...
      Windowing Words : ['enter' 'gate' 'making' 'wherever' 'wish']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 9/18
    Target Word : gate
    - Windowing...
      Windowing Words : ['enter' 'gate' 'making' 'obeisance' 'wish']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 10/18
    Target Word : making
    - Windowing...
      Windowing Words : ['enter' 'gate' 'making' 'obeisance' 'say']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget

       - Target Word Vector...
  # Terget Word 12/23
    Target Word : knew
    - Windowing...
      Windowing Words : ['drinking' 'knew' 'place' 'spring' 'tribe']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 13/23
    Target Word : drinking
    - Windowing...
      Windowing Words : ['drinking' 'eat' 'knew' 'place' 'tribe']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 14/23
    Target Word : place
    - Windowing...
      Windowing Words : ['drink' 'drinking' 'eat' 'knew' 'place']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 15/23
    Target Word : eat
    - Windowing...
      Windowing Words : ['drink' 'drinking' 'eat' 'place' 'provision']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # T

       - Target Word Vector...
  # Terget Word 22/43
    Target Word : better
    - Windowing...
      Windowing Words : ['better' 'enter' 'exchange' 'said' 'worse']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 23/43
    Target Word : worse
    - Windowing...
      Windowing Words : ['better' 'city' 'enter' 'exchange' 'worse']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 24/43
    Target Word : enter
    - Windowing...
      Windowing Words : ['ask' 'better' 'city' 'enter' 'worse']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 25/43
    Target Word : city
    - Windowing...
      Windowing Words : ['abasement' 'ask' 'city' 'enter' 'worse']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terg

       - Target Word Vector...
  # Terget Word 12/17
    Target Word : shall
    - Windowing...
      Windowing Words : ['day' 'good' 'lord' 'reward' 'shall']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 13/17
    Target Word : reward
    - Windowing...
      Windowing Words : ['fear' 'good' 'lord' 'reward' 'shall']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 14/17
    Target Word : lord
    - Windowing...
      Windowing Words : ['fear' 'lord' 'reward' 'shall']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 15/17
    Target Word : fear
    - Windowing...
      Windowing Words : ['fear' 'grieve' 'lord' 'reward' 'shall']
    - Weighting...
       - Count Overlaps...
       - Normalized Concept...
       - Target Word Vector...
  # Terget Word 16/17
    Ta

OSError: [Errno 28] No space left on device

In [27]:
# import os

# scores = {} # scores is an empty dict already
# target = "./assets/realTrainLeskModel_1_100.bin.txt"
# if os.path.getsize(target) > 0:      
#     with open(target, "rb") as f:
#         unpickler = pickle.Unpickler(f)
#         # if file is not empty scores will be equal
#         # to the value unpickled
#         scores = unpickler.load()

# scores






# # readFileBin("./assets/realTrainLeskModel_1_100.bin.txt")

MemoryError: 

# **Classification (Semantic Naive Bayes) - LESK**

**Hitung probability FINAL untuk tiap kelas**

In [ ]:
def calculateProbabilityDataTest(probClass, testDataInClass):
    # sum all cell
    sumAllCell = np.ravel(testDataInClass.values).sum()
    # productAllCell = np.prod(testDataInClass.values)

    # jumlahkan dengan probability class-nya
    classProbForTest = np.log(probClass) + sumAllCell
    # classProbForTest = probClass * productAllCell

    return classProbForTest

**Mapping data test dengan fitur di data train**

In [ ]:
def mapDataToClass(test_data, superMatrixClass):
    # ambil token ayat dari test data
    tokenAyat = test_data['token_ayat']

    # ambil super matrix dengan index/terms yang ada di tokenAyat
    superMatrixTest = superMatrixClass.loc[tokenAyat]

    return superMatrixTest

**Hitung Super Matriks dan Probability Tiap Cell-nya untuk Data dengan kelas tertentu**

In [ ]:
def calculateLogCell(cellValue):
    if cellValue == 0:
        return cellValue
    # hasil log(0.__) akan bernilai negatif, itu tidak apa-apa, hasilnya sama saja
    return np.log(cellValue)

In [ ]:
'''============ 1. ============'''
''' 
### DONE Date: 25/10/2020 ###
Input   : dataPerClass
Output  : super matrix dari kelas tersebut
Problem : -
''' 
def createSuperMatrix(dataPerClass):
    # kalo datanya kosong untuk salah satu kelas-nya
    if len(dataPerClass) == 0:
        return 'empty'

    # inisialisasi df kosong untuk dataPerClass
    # ini bisa error kalo datanya kosong untuk salah satu kelas-nya, karena ada ini "dataPerClass[0]" > makanya cek dulu diatas
    dfAccumulatorDataPerClass = pd.DataFrame(0.0, columns=dataPerClass[0]["weightedAyat"].columns, index=dataPerClass[0]["weightedAyat"].index)
    # inisiasi variable untuk total penjumlahan cell keseluruhan pada class tersebut
    accumulatorAllCells = 0
    
    # looping untuk tiap data di class tersebut
    for i, data in enumerate(dataPerClass):
        print("data ke - {} / {}".format(i, len(dataPerClass)-1))
        # jumahkan tiap cells di seluruh doc yang classnya sama rumusnya yang W*(t_i, s_j, c) -> matriks
        dfAccumulatorDataPerClass = dfAccumulatorDataPerClass.add(data["weightedAyat"], fill_value=0)
    
    # jumlahkan keseluruhan cells di seleuruh doc yang classnya sama sigma_t . sigma_s W*(t, s, c) + |V||S| -> float 
    accumulatorAllCells += (dfAccumulatorDataPerClass.to_numpy().sum() / len(dataPerClass)) + (len(dataPerClass[0]['weightedAyat'].index) * len(dataPerClass[0]['weightedAyat'].columns))

    # bagi setiap cell-nya dengan jumlah data train di kelas itu
    dfAccumulatorDataPerClass = dfAccumulatorDataPerClass.div(len(dataPerClass))

    # bagi lagi tiap cell-nya dengan nilai accumulator | Pr(M_ij|c) + 1 -> rumus probability cell
    dfAccumulatorDataPerClass = dfAccumulatorDataPerClass.add(1).div(accumulatorAllCells)

    # '''GAK PAKE LOG DULU'''
    # log-kan setiap cell-nya | logPr(M_ij|c) -> rumus prob. cell pakai log
    dfAccumulatorDataPerClass = dfAccumulatorDataPerClass.applymap(calculateLogCell)
    # dari bentuk tuple, rubah lagi ke dataframe, agar supaya gampang ngolahnya
    dfAccumulatorDataPerClass = pd.DataFrame(dfAccumulatorDataPerClass)
        
    return dfAccumulatorDataPerClass

**Fungsi Utama Predict Test Data**

In [ ]:
def checkZero(arr):
    for i in arr:
        if i == 0:
            return True
    return False

# checkZero([1,2,4,3, 0])

'''============ 6. ============'''
''' 
### DONE Date: 25/10/2020 ###
Input   : train data & test data
Output  : hasil prediksi data test
Problem : -
''' 
def createProbabilityModel(listPredictionLabelTest, currentBinaryLabel, class0_train_data, class1_train_data, test_data, probClass0, probClass1):
    predictionLabelTest = list()
    # predictionLabelTest['label_number'] = currentBinaryLabel

    # |c| jumlah data belonging to the class c
    totalDocClass0 = len(class0_train_data)
    totalDocClass1 = len(class1_train_data)
    
    # buat super matrix dan hitung probability tiap cell-nya | sampai ke rumus logPr(M_ij|c)
    print("== creating super matrix class 0... ==")
    superMatrixClass0 = createSuperMatrix(class0_train_data) 
    print("== creating super matrix class 1... ==")
    superMatrixClass1 = createSuperMatrix(class1_train_data)

    # display(superMatrixClass1)
    # print(checkZero(np.ravel(superMatrixClass1.values)))

    # looping untuk tiap-tiap testing data
    for i, dTest in enumerate(test_data):
        # mapping fitur data train ke data testing
        if type(superMatrixClass0) == str:
            # kalau matrix kelas 0 nya ga ada, otomatis di prediksi ke kelas 1
            predictionLabelTest.append(1)
            # predictionLabelTest[str(i)] = 1
            # listPredictionLabelTest.append(predictionLabelTest)
            continue # lanjut ke next data test
        elif type(superMatrixClass1) == str:
            # kalau matrix kelas 1 nya ga ada, otomatis di prediksi ke kelas 0
            predictionLabelTest.append(0)
            # predictionLabelTest[str(i)] = 0
            # listPredictionLabelTest.append(predictionLabelTest)
            continue # lanjut ke next data test

        testDataInClass0 = mapDataToClass(dTest, superMatrixClass0)
        testDataInClass1 = mapDataToClass(dTest, superMatrixClass1)

        print("Testing Data ke : ", i+1)
        display(testDataInClass0)
        display(testDataInClass1)

        print("NILAI PROBABILITY")
        # hitung nilai probability untuk tiap kelasnya
        probabilityClass0 = calculateProbabilityDataTest(probClass0, testDataInClass0) 
        probabilityClass1 = calculateProbabilityDataTest(probClass1, testDataInClass1)

        # tampung prob di list
        listProb = [probabilityClass0, probabilityClass1]

        print("Nilai log(Prob) untuk Kelas 0 : ", probabilityClass0)
        print("Nilai log(Prob) untuk Kelas 1 : ", probabilityClass1)
        print("Max Value        : ", max(listProb))
        
        # tentuin classnya berdasarkan nomor indexnya
        classPrediction = listProb.index(max(listProb))
        predictionLabelTest.append(classPrediction)
        # predictionLabelTest[i] = classPrediction
        print("Label Prediction : ", classPrediction)

    listPredictionLabelTest.append(predictionLabelTest)

    return listPredictionLabelTest

**Hitung Probilitas Untuk Kelas 0 dan 1**

In [ ]:
'''============ 7. ============'''
''' 
### DONE Date: 25/10/2020 ###
Input   : class0_train_data, class1_train_data, number_train_data
Output  : class prob untuk tiap kelasnya
Problem : -
''' 
def calculateClassProbability(number_class0_train_data, number_class1_train_data, number_train_data):
    # probability class 0
    probClass0 = number_class0_train_data / number_train_data
    # probability class 1
    probClass1 = number_class1_train_data / number_train_data
        
    return probClass0, probClass1

**Pisahkan Data Train dan Test**

In [ ]:
'''============ 8. ============'''
''' 
### DONE Date: 25/10/2020 ###
Input   : train data
Output  : train data yang telah dikelompokkan berdasarkan kelasnya
Problem : -
''' 
def devideTrainDataByClass(train_data, currentBinaryLabel):
    class0_train_data, class1_train_data = list(), list()
    
    for i, data in enumerate(train_data):
        if data["labelsPerAyat"][currentBinaryLabel] == 0:
            class0_train_data.append(data)
        else:
            class1_train_data.append(data)
    
    return class0_train_data, class1_train_data

**Split Data Train & Test**
```
# Pembagiannya masih untuk DUMMY
```

In [ ]:
'''============ 9. ============'''
''' 
### DONE Date: 22/10/2020 ###
Input   : listWeightedAyat
Output  : train data & test data
Problem : -
''' 
def splitData(listWeightedAyat):
    # Shuffle ayat agar yang dijadikan data test adalah random
#    random.shuffle(listWeightedAyat)

    # ambil train data
    train_data = listWeightedAyat[:14]

    # ambil data terjemahannya dulu dari sheet
    listAyatInString = pd.read_excel(datapath, sheet_name='proceed-data').Terjemahan.values[13:]
    labelsAllAyat = pd.read_excel(datapath, sheet_name='proceed-data').iloc[:,4:20].values[13:]

    # list test data
    test_data = []
    for i, ayat in enumerate(listAyatInString):
        test_data.append({
          'token_ayat' : ast.literal_eval(listAyatInString[i]),
          'label_ayat' : labelsAllAyat[i]
        })
    
    return train_data, test_data

**Run Classification**

```
# Bagaimana kalau data trainnya hanya punya 1 label, hanya punya kelas 0 atau kelasnya 1 saja?
```

In [ ]:
leskModel = readFileBin(leskOutputPath)
leskModel

#listWeightedAyat = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# load data terms-by-concept matriks dari file bin
listWeightedAyat = readFileBin(leskOutputPath)
# panggill fungsi main / entar pake ini
#mainClassification(listWeightedAyat)
train_data, test_data = splitData(listWeightedAyat)
# nama label untuk dataset
labelsName = list(range(16))
# inisialisasi dict untuk prediction result
listPredictionLabelTest = list()


# ALTERNATIF 1
for currentBinaryLabel in labelsName:
  # kelompokkan data train berdasarkan kelasnya
  class0_train_data, class1_train_data = devideTrainDataByClass(train_data, currentBinaryLabel)

  # hitung probabilitas per kelasnya
  probClass0, probClass1 = calculateClassProbability(len(class0_train_data), len(class1_train_data), len(train_data))

  # buat model probability menggunakan SNB untuk currentBinaryLabel
  listPredictionLabelTest = createProbabilityModel(listPredictionLabelTest, currentBinaryLabel, class0_train_data, class1_train_data, test_data, probClass0, probClass1)
  # listPredictionLabelTest.append(predictionLabelTest)

listPredictionLabelTest



# # ALTERNATFIF 2
# # ambil label ke-2 | diitung dari 0
# currentBinaryLabel = 0 
# # kelompokkan data train berdasarkan kelasnya
# class0_train_data, class1_train_data = devideTrainDataByClass(train_data, currentBinaryLabel)

# # hitung probabilitas per kelasnya
# probClass0, probClass1 = calculateClassProbability(len(class0_train_data), len(class1_train_data), len(train_data))

# # buat model probability menggunakan SNB untuk currentBinaryLabel
# listPredictionLabelTest = createProbabilityModel(listPredictionLabelTest, currentBinaryLabel, class0_train_data, class1_train_data, test_data, probClass0, probClass1)
# # listPredictionLabelTest.append(predictionLabelTest)

# listPredictionLabelTest

**Rapihkan Label Prediksi**

In [ ]:
# banyak data test
listLenTest = list(range(len(test_data)))

# simpan di pandas dataframe
dfLabelPred = pd.DataFrame(listPredictionLabelTest, columns=listLenTest)

# lakukan transpose, biar bentuknya sama dengan data di excel
dfLabelPred = dfLabelPred.T

dfLabelPred

**Siapkan Label Aktual dari Data Test**

In [ ]:
actualLabel = []
for test in test_data:
    actualLabel.append(test['label_ayat'])

dfActualLabel = pd.DataFrame(actualLabel)

dfActualLabel# **Classification (Semantic Naive Bayes) - LESK**

# **Evaluation**

In [ ]:
def hamming_loss(label_pred, label_real, len_data, jum_label):
    count_salah = 0
    for i, pred in enumerate(label_pred):
        for j, angka in enumerate(pred):
            if label_pred[i][j] == label_real[i][j]:
                continue
            else:
                count_salah += 1
    hamming_loss = (1/len_data)*(1/jum_label)*count_salah
    print("Jumlah Salah  : ", count_salah)
    return hamming_loss

In [ ]:
hamming = hamming_loss(dfLabelPred.values.tolist(), dfActualLabel.values.tolist(), len(test_data), 16)
print("Nilai Hamming : ", hamming)